# Convolutional Neural Networks For Image Recognition

## Setup and Imports

**Import Packages**

In [ ]:
import keras as kr
import numpy as np
import tensorflow as tf

from keras.datasets import cifar100

from sklearn.model_selection import train_test_split

from random import randint
import time

## Import and Preprocess Dataset

**Import Cifar100 Data Set**

In [ ]:
(X, y), (x_test, y_test) = cifar100.load_data(label_mode='fine')
X = X[:20]
y = y[:20]

In [ ]:
################################################################################
#    Title: One Hot Encoding
################################################################################
#    Description: 
#        This function extends a matrix to one-hot encoding
#    
#    Parameters:
#        y    Array of label values
# 
#    Returns:
#        y_new    One hot encoded array of labels
################################################################################
def one_hot(y):
    n_values = np.max(y) + 1
    y_new = np.eye(n_values)[y[:,0]]
    return y_new

In [ ]:
y=one_hot(y)
y_test=one_hot(y_test)

**Resize Images to be compatible with VGG16**

In [ ]:
with tf.name_scope('ZCA'):
    flatx = tf.cast(tf.reshape(X, (-1, np.prod(X.shape[-3:])),name="reshape_flat"),tf.float64,name="flatx")
    sigma = tf.tensordot(tf.transpose(flatx),flatx, 1,name="sigma") / tf.cast(tf.shape(flatx)[0],tf.float64)
    s, u, v = tf.svd(sigma,name="svd")
    net = tf.tensordot(tf.tensordot(u,tf.diag(1. / tf.sqrt(s+epsilon)),1,name="inner_dot"),tf.transpose(u),1, name="pc")
    net = tf.tensordot(flatx, net,1,name="whiten")
    net = tf.reshape(net,np.shape(x_batch), name="output")
with tf.Session() as sess:
        zca[ind:end] = sess.run(net)

In [ ]:
################################################################################
#    Title: Preprocess Images
################################################################################
#    Description: 
#        This function resizes 32x32x3 images to 128x128x3 by adding padding
#    
#    Parameters:
#        X            Array of 32x32x3 Images
#        num_batch    Number of batches to do the computation
# 
#    Returns:
#        A 224x224 set of images
################################################################################
def preproc_images(X, num_batch):
    l = len(X)
    d=int(l/num_batch)
    X_new = np.zeros((l,224,224,3)).astype(np.int8)
    ind = 0
    for i in range (num_batch):
        end = ind+d
        if i==num_batch-1:
            end = l;
        x_batch = X[ind:end]
        net = tf.image.resize_images(x_batch, size=(256,256))
        net = tf.image.resize_image_with_crop_or_pad(net, 224, 224)
        with tf.Session() as sess:
                X_new[ind:end] = sess.run(net)
        ind = ind + d
    return X_new

In [ ]:
################################################################################
#    Title: ZCA
################################################################################
#    Description: 
#        This function applies ZCA Whitening to the image set
#    
#    Parameters:
#        X            Array of MxNxC images
#        num_batch    Number of batches to do the computation
# 
#    Returns:
#        An array of MxNxC zca whitened images
################################################################################
def zca(X, num_batch, epsilon=1e-5):
    l,M,N,C = np.shape(X)
    d=int(l/num_batch)
    zca = np.zeros((l,M,N,C))
    ind = 0
    for i in range (num_batch):
        end = ind+d
        if i==num_batch-1: end = l;
        x_batch = X[ind:end]
        
        with tf.name_scope('ZCA'):
            flatx = tf.cast(tf.reshape(x_batch, (-1, np.prod(x_batch.shape[-3:])),name="reshape_flat"),tf.float64,name="flatx")
            sigma = tf.tensordot(tf.transpose(flatx),flatx, 1,name="sigma") / tf.cast(tf.shape(flatx)[0],tf.float64)
            s, u, v = tf.svd(sigma,name="svd")
            net = tf.tensordot(tf.tensordot(u,tf.diag(1. / tf.sqrt(s+epsilon)),1,name="inner_dot"),tf.transpose(u),1, name="pc")
            net = tf.tensordot(flatx, net,1,name="whiten")
            net = tf.reshape(net,np.shape(x_batch), name="output")
        with tf.Session() as sess:
                zca[ind:end] = sess.run(net)
        ind = end
    return zca

In [ ]:
zca(X,2)

In [ ]:
time1 = time.time()
X = preproc_images(X,4)
time2 = time.time()
print('Time Elapsed - Resizing: '+str(time2-time1));
X = zca(X,4)
time3 = time.time()
print('Time Elapsed - ZCA Whitening: '+str(time3-time2));

**Split Training set into Training and Validation sets**

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=.1, random_state=0)

## Coarse Training

**Define Global Variables**

In [ ]:
# The number of coarse categories
coarse_categories = 20

# The number of fine categories
fine_categories = 100

# The threshold percentage in thresholding layer
sigma = .01

**Import VGG16 Pretrained on Imagenet**

In [ ]:
from keras.applications.vgg16 import VGG16

Citation credit for VGG16 model to:

Simonyan, Karen, and Andrew Zisserman. “Very Deep Convolutional Networks for Large-Scale Image Recognition.” [1409.1556] Very Deep Convolutional Networks for Large-Scale Image Recognition, 10 Apr. 2015, arxiv.org/abs/1409.1556.

In [ ]:
from keras.layers import Input, Dense, Conv3D
from keras.models import Model
in_layer = Input(shape=(224, 224, 3), dtype='float32', name='main_input')
model = VGG16(include_top=True, weights='imagenet', input_tensor=in_layer, input_shape=(224, 224, 3))

**Modify Model for Cifar100**

In [ ]:
out_coarse = Dense(fine_categories, activation='softmax')(model.layers[-2].output)
model = Model(input=in_layer,output=out_coarse)
model.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
time1 = time.time()
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(featurewise_center=True,
    featurewise_std_normalization=True,
    zca_whitening=True,
    zca_epsilon=1e-6,
    rotation_range=20,
    width_shift_range=0.4,
    height_shift_range=0.4,
    horizontal_flip=True,
    vertical_flip=True)

datagen.fit(x_train)
time2 = time.time()
print(time2-time1)

**Train Model**

In [ ]:
index= 0
step = 5
stop = 300

while index < stop:
    model.fit(x_train, y_train, batch_size=32, initial_epoch=index, epochs=index+step, validation_data=(x_val, y_val))
    index += step
    model.save_weights('data/models/model_coarse_'+str(index))

## Fine-Tuning

In [ ]:
stop = 5
model.load_weights('data/models/model_coarse_'+str(stop))

In [ ]:
trainable_index = 17

for i in range(len(model.layers)):
    if i<trainable_index:
        model.layers[i].trainable=False

In [ ]:
def fine_model():
    out_fine = Dense(fine_categories, activation='softmax')(model.layers[-2].output)
    model_fine = Model(input=in_layer,output=out_fine)
    model_fine.compile(optimizer= 'adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model_fine

In [ ]:
fine_models = {'models' : [{} for i in range(coarse_categories)]}
print(fine_models['models'])
for i in range(coarse_categories):
    model_i = fine_model()
    fine_models['models'][i] = model_i
print(fine_models['models'])

In [ ]:
index = np.where(np.array(y_train)[:,1] == 1)
y_i = [y_train[j] for j in index]
x_i = [x_train[j] for j in index]
print(np.shape(y_i[0]))
print(np.shape(x_i[0]))

In [ ]:
for i in range(coarse_categories):
    print("Training Fine Classifier: ", str(i))
    
    index= 0
    step = 5
    stop = 5

    ind = np.where(np.array(y_train)[:,i] == 1)
    y_i = [y_train[j] for j in ind]
    x_i = [x_train[j] for j in ind]
    
    indv = np.where(np.array(y_val)[:,i] == 1)
    y_iv = [y_val[j] for j in indv]
    x_iv = [x_val[j] for j in indv]
    
    print(np.shape(x_iv))
    while index < stop:
        fine_models['models'][i].fit(x_i, y_i, batch_size=2, initial_epoch=index, epochs=index+step, validation_data=(x_iv, y_iv))
        index += step
        fine_models['models'][i].save_weights('data/models/model_fine_'+str(i))

In [ ]:
# Calculate probabalistic averaging layer and coarse probabilities
prob_ave, coarse_prob = calc_prob_ave(output_c, ck)

# Create new independent layers for each coarse category
for i in range(20):
    #thresholded = Threshold(coarse_prob,i)(net_shared) - Threshold layer not functional
    net = indep_layers(net_shared, str(i+1))
    fine_output = Dense(5, activation='softmax')(net)
    
    if i==0: output_f = fine_output
    else: output_f = Concatenate(axis=1)([output_f, fine_output])


# Apply probabalistic averaging layer to the result of the fine predictions
weighted = Multiply()([output_f, prob_ave])

**Compile new network**

In [ ]:
model = Model(inputs=[base_model.input, ck], 
              outputs=weighted)

model.compile(optimizer= 'adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

**Load Coarse-To-Fine Mappings**

In [ ]:
# Eventually will want to do spectral clustering for coarse to fine mappings
# currently just using coarse categories defined by cifar

##### REPLACE WITH PRE_BUILT ONE HOT ENCODER #####
# Get Coarse to Fine Mappings
C2K = np.loadtxt('C2K.txt', dtype=int)
C2K_dot = np.zeros((20,100))
for i in range(len(C2K)):
    C2K_dot[C2K[i], i] = 1

In [ ]:
##### REPLACE WITH NP.REPEAT #####
C2K_train = np.zeros((len(x_train), len(C2K_dot), len(C2K_dot[0])))
C2K_val = np.zeros((len(x_val), len(C2K_dot), len(C2K_dot[0])))
C2K_test = np.zeros((len(x_test), len(C2K_dot), len(C2K_dot[0])))

for i in range(len(x_train)):
    C2K_train[i] = C2K_dot

for i in range(len(x_val)):
    C2K_train[i] = C2K_dot

for i in range(len(x_test)):
    C2K_train[i] = C2K_dot

**Load Coarse Weights**

In [ ]:
for layer in base_model.layers:
    # Save layer name for readability
    layer_name = layer.get_config()['name']
    
    # Set all shared layers to not be trainable
    if 'shared_layer' in layer_name:
        layer.trainable = False
    
    # Load weights for new independent layers
    if 'indep_layer' in layer_name:
        seg = layer_name.split('__')[1]
        for f_layer in model.layers:
            if seg in f_layer.get_config()['name']:
                f_layer.set_weights(layer.get_weights())

**Fit Training Data**

In [ ]:
index= 0
step = 5
stop = 50

while index < stop:
    model.fit([x_train, C2K_train], y_train, initial_epoch=index, epochs=index+step, batch_size=256, \
             validation_data=([x_val, C2K_val],y_val))
    model.save_weights('data/models/layer_'+str(index))
    index += step

**Evaluate on testing set**

In [ ]:
ct = np.zeros((20,len(x_test),100))
for i in range(len(x_test)):
    for j in range(20):
        ct[j,i] = C2K_dot[j]
        
model.evaluate([x_test, C2K_test], y_test, batch_size=64, verbose=1)